In [9]:
import sys
import json
import string
import torch
import numpy as np
from torch.utils.data import DataLoader

if r'G:\PythonProjects\WineRecognition2' not in sys.path:
    sys.path.insert(0, r'G:\PythonProjects\WineRecognition2')

from nn.utils import generate_tag_to_ix, get_model_confidence, CustomDataset
from nn.mlflow_utils import log_mlflow_on_test
from data_master import DataGenerator, count_unk_foreach_tag

In [10]:
MODEL_NAME = 'BiLSTM_CRF'
MODEL_PATH = 'G:/PythonProjects/WineRecognition2/artifacts/train/BiLSTM_CRF_10112021_032950/model/data/model.pth'
RUN_NAME = 'Test-100-256'
START_TIME = ''
OUTPUT_DIR = 'G:/PythonProjects/WineRecognition2/artifacts/test/test'
DATA_PATH = r'G:\PythonProjects\WineRecognition2\data\text\menu_txt_tagged.txt'
VOCAB_PATH = 'G:/PythonProjects/WineRecognition2/data/vocabs/Words_Halliday_Wine_AU_WORD_NUMS.json'
DATAINFO_PATH = 'G:/PythonProjects/WineRecognition2/data_info.json'
COMPUTE_METRICS = True
CASE_SENSITIVE_VOCAB = True
USE_NUM2WORDS = False
DEVICE = 'cpu'

In [11]:
with open(VOCAB_PATH, 'r', encoding='utf-8') as file:
    word_to_ix = json.load(file)
len(word_to_ix)

12172

In [12]:
with open(DATAINFO_PATH) as file:
    keys = json.load(file)['keys']['all']
    
if not COMPUTE_METRICS:
    keys.append('UNKNOWN')
    
tag_to_ix = generate_tag_to_ix(keys)
ix_to_tag = {value: key for key, value in tag_to_ix.items()}
tag_to_ix

{'Add_TradeName': 0,
 'Add_Brand': 1,
 'Add_KeyWordTrue': 2,
 'Add_KeyWordFalse': 3,
 'Add_GrapeVarieties': 4,
 'Add_GeoIndication': 5,
 'Add_WineType': 6,
 'Add_BottleSize': 7,
 'Add_Sweetness': 8,
 'Add_WineColor': 9,
 'Add_ClosureType': 10,
 'Add_Certificate': 11,
 'Add_Vintage': 12,
 'Add_Price': 13,
 'Punctuation': 14,
 'Other': 15}

In [13]:
with open(DATA_PATH, encoding='utf-8') as file:
    x_test = DataGenerator.generate_sents2(file.read().split('\n'))

    
dataset = CustomDataset(
    x_test,
    tag_to_ix,
    word_to_ix,
    case_sensitive=CASE_SENSITIVE_VOCAB,
    convert_nums2words=USE_NUM2WORDS
)

y_test = [tags for _, tags in dataset.raw_data()]
dataloader = DataLoader(dataset, batch_size=2048, shuffle=False, drop_last=False)

In [ ]:
model = torch.load(MODEL_PATH).to(DEVICE)
model.eval()

In [ ]:
y_pred = []
with torch.no_grad():
    for x, tags, mask, _ in dataloader:
        x = x.to(DEVICE)
        tags = tags.to(DEVICE)
        mask = mask.to(DEVICE)
        best_tag_seq = model(x, mask)
        y_pred.extend(best_tag_seq)

In [ ]:
x_tensor = [
    torch.tensor(dataset.sentence_to_indices(sentence), dtype=torch.int64) for sentence, _ in dataset.raw_data()
]

In [ ]:
confs = get_model_confidence(model, x_tensor, DEVICE)

In [ ]:
unk_foreach_tag = count_unk_foreach_tag(x_tensor, y_test, list(tag_to_ix), dataset.word_to_ix[dataset.unk])

In [ ]:
for index, pred in enumerate(y_pred):
    y_pred[index] = [ix_to_tag[tag] for tag in pred]

In [ ]:
run_params = {
    'model_name': MODEL_NAME,
    'model_path': MODEL_PATH,
    'run_name': RUN_NAME,
    'start_time': START_TIME,
    'output_dir': OUTPUT_DIR,
    'data_path': DATA_PATH,
    'vocab_path': VOCAB_PATH,
    'datainfo_path': DATAINFO_PATH,
    'case_sensitive_vocab': CASE_SENSITIVE_VOCAB,
    'device': DEVICE,
    'models_confidence': np.mean(confs),
    'compute_metrics': COMPUTE_METRICS,
    'unk_foreach_tag': json.dumps(unk_foreach_tag),
    'use_num2words': USE_NUM2WORDS
}

In [ ]:
log_mlflow_on_test(
    run_params,
    classes=list(ix_to_tag.values()),
    x_test=[sentence for sentence, _ in dataset.raw_data()],
    y_pred=y_pred,
    y_true=y_test
)